# Slot 4: Fine-Tuning

It is divided into X parts

1. Preparations
2. The dataset
    - Loading data
    - Augmenting data
4. Fine-tuning routine
5. LORA - Benefits and usage

1. Other showcases
2. Fine-tuning (follow steps from slides)
3. Show data set
4. Show model
5. Show and explain training parameters
6. Fine-tune
7. Result

### Preparations
Start with installing the packages necessary for the task

In [ ]:
%pip install -r requirements.txt

Import packages and load environment

In [ ]:
import torch
from datasets import load_dataset
from transformers import ( 
    AutoTokenizer, 
    AutoModelForCausalLM,
    TrainingArguments,
    pipeline,
    logging,
    )
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

#### Load Model

We will be using Llama 3.2 with 1 billion parameters. **Beware that you need to request access to it beforehand.**

In [ ]:
#load tokenizer and model with QLoRA config
# compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit = use_4bit,
#     bnb_4bit_quant_type = bnb_4bit_quant_type,
#     bnb_4bit_compute_dtype = compute_dtype,
#     bnb_4bit_use_double_quant = use_nested_quant,)

#checking GPU compatibility with bfloat16
# if compute_dtype == torch.float16 and use_4bit:
#     major, _ = torch.cuda.get_device_capability()
#     if major >= 8:
#         print("="*80)
#         print("Your GPU supports bfloat16, you are getting accelerate training with bf16= True")
#         print("="*80)


# Model to be used
model_name = "meta-llama/Llama-3.2-1B-Instruct" # https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct

#Load LLama tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#load the entire model on the GPU
device_map = {"":0}
#load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config = bnb_config,
    device_map = device_map,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

### Showcase: Base model

Here we check the result of the base model on our query.
We use the text generation pipeline to create a SPARQL query on the DBLP knowledge graph.

In [ ]:
# Run text generation pipeline with our next model
prompt = "Show the Wikidata ID of the person Robert Schober. His entity ID is <https://dblp.org/pid/95/2265>. Provide only a SPARQL query as answer."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

### Showcase: Fine-tuning

First, we load the data set to be used.

In [ ]:
dataset_name = "awalesushil/DBLP-QuAD" # https://huggingface.co/datasets/awalesushil/DBLP-QuAD

#load dataset
dataset = load_dataset(dataset_name,split = "train")

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['question'])):
        text = f"### Question: {example['question'][i]}\n ### Answer: {example['query'][i]}"
        output_texts.append(text)
    return output_texts

response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

Then we set all parameters necessary for fine-tuning

In [ ]:
#output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

#number of training epochs
num_train_epochs = 5

#enable fp16/bf16 training (set bf16 to True when using A100 GPU in google colab)
fp16 = False
bf16 = True

#batch size per GPU for training
per_device_train_batch_size = 4

#batch size per GPU for evaluation
per_device_eval_batch_size = 4

#gradient accumulation steps - No of update steps
gradient_accumulation_steps = 1

#learning rate
learning_rate = 2e-4

#weight decay
weight_decay = 0.001

#Gradient clipping(max gradient Normal)
max_grad_norm = 0.3

#optimizer to use
optim = "paged_adamw_32bit"

#learning rate scheduler
lr_scheduler_type = "cosine"

#seed for reproducibility
seed = 1

#Number of training steps
max_steps = -1

#Ratio of steps for linear warmup
warmup_ratio = 0.03

#group sequnces into batches with same length
group_by_length = True

#save checkpoint every X updates steps
save_steps = 0

#Log at every X updates steps
logging_steps = 50

#### PEFT Config

As we use LORA, we need to set up some additional parameters

In [ ]:
# LORA
lora_r = 64 #lora attention dimension/ rank
lora_alpha = 16 #lora scaling parameter
lora_dropout = 0.1 #lora dropout probability

# BitsAndBytes for QLORA
# use_4bit = True
# bnb_4bit_compute_dtype = "float16"
# bnb_4bit_quant_type = "nf4"
# use_nested_quant = False

#Load LoRA config
peft_config = LoraConfig(
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    r  = lora_r,
    bias = "none",
    task_type = "CAUSAL_LM",
)

### Training

In [ ]:
#Set Training parameters
training_arguments = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size = per_device_train_batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    optim = optim,
    save_steps = save_steps,
    logging_steps = logging_steps,
    learning_rate = learning_rate,
    # fp16 = fp16,
    # bf16 = bf16,
    max_grad_norm = max_grad_norm,
    weight_decay = weight_decay,
    lr_scheduler_type = lr_scheduler_type,
    warmup_ratio = warmup_ratio,
    group_by_length = group_by_length,
    max_steps = max_steps,
    report_to = "tensorboard",
)


#maximum sequence length to use
max_seq_length = 1024 # used to be None
packing = False # Same as default
#SFT Trainer
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    peft_config = peft_config,
    # dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = training_arguments,
    tokenizer = tokenizer,
    packing = False,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)

#Start training
trainer.train()

#### Saving the model and Testing
Use the text generation pipeline to ask questions like “What is Machine Learning? etc.” Note that I’m formatting the input to match Llama 2 prompt template.

In [ ]:
new_model = f"{model_name}-finetuned-DBLP-QUAD2" # name of the fine-tuned model

#save trained model
trainer.model.save_pretrained(f"results/finetuned models/{new_model}")

In [ ]:
model = AutoModelForCausalLM.from_pretrained(f'results/finetuned models/{new_model}', device_map=device_map)
# Run text generation pipeline with our next model
prompt = "Show the Wikidata ID of the person Robert Schober. Write a SPARQL query. His entity ID is <https://dblp.org/pid/95/2265>."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"### Question: {prompt}")
print(result[0]['generated_text'])